In [ ]:
import variational_bayes as vb
import numpy as np
from scipy.special import expit
from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
# Define parameters
np.random.seed(1)
num_samples = 10000
num_dims = 3
bias = -8

# Generate data
features = np.hstack([
    np.ones((num_samples, 1)), 
    np.random.normal(0, 1, (num_samples, num_dims))
])
theta = np.concatenate([[bias], np.random.normal(0, 1, num_dims)])
predictor = np.dot(features, theta)
proba = expit(predictor)

x = np.random.uniform(0, 1, proba.shape) < proba
print("Mean prevalence: %s" % np.mean(x))

In [ ]:
# Define the factors
q_theta = vb.MultiNormalDistribution(
    np.random.normal(0, 1e-3, num_dims + 1),
    np.diag(1e-3 * np.ones(num_dims + 1))
)
# Define the helper parameters
q_xi = vb.DeltaDistribution(np.abs(np.random.normal(0, 1e-3, num_samples)))

# Define likelihoods (delta distriutions do not need priors)
likelihoods = [
    vb.LogisticLikelihood(x, features, q_theta, q_xi),
    vb.NormalLikelihood(q_theta, 0, 1e-3),
]

model = vb.Model({'theta': q_theta, 'xi': q_xi}, likelihoods)

In [ ]:
# Optimize the evidence lower bound
elbo, _ = model.update(
    None, convergence_predicate=vb.ConvergencePredicate(1e-4, 10)
)
plt.plot(elbo)

In [ ]:
vb.plot_comparison(model['theta'], theta)